In [15]:
import re
import csv
import calendar
import pytz
import pandas as pd
import requests as req

from bs4 import BeautifulSoup as bs
from datetime import datetime as dt

print("Success Import Library!")

Success Import Library!


In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
local_timezone = pytz.timezone('Asia/Jakarta')
time_now = dt.now(local_timezone).strftime("%Y-%m-%d %H:%M:%S")
file_path = "/content/drive/MyDrive/Data Engineering/"

In [13]:
def format_tanggal(month, year, long_time, day = 1):
    test_date = dt.strptime(f"{year}-{month}-{day}", "%Y-%m-%d")
    date_generated = pd.date_range(test_date, periods=long_time)

    return date_generated

def source_detik(mm, yy, long_time2):

    # untuk membuat files csv dan menulis column header
    with open(file_path + f"result_scrape_{time_now}.csv", "a") as file:
        writer = csv.writer(file, delimiter=",")
        writer.writerow(['title', 'released', 'url', 'content'])
        count = 0

        date_generate = format_tanggal(mm, yy, long_time2)
        formattgl = date_generate.strftime("%d-%m-%Y")

        for tgl in formattgl:
            url = f"https://news.detik.com/indeks?date={tgl}"
            get_url = req.get(url).text
            soup = bs(get_url, 'lxml')

            try:
                paging = soup.find_all('div', 'pagination text-center mgt-16 mgb-16')[0].find_all('a')[-2]
                last_page = paging.text
            except:
                last_page = 1

            for page in range(1, int(last_page) + 1):
                try:
                    url2 = f"https://news.detik.com/indeks/{page}?date={tgl}"
                    get_url2 = req.get(url2).text
                    soup2 = bs(get_url2, "lxml")
                    contai = soup2.find_all('div', class_='media__text')

                    # scrap tgl, judul, url berita
                    for x in contai:
                            time = x.find('div', class_='media__date').find('span')['title'].split()
                            tgl_ = f"{time[1]}/{time[2]}/{time[3]} {time[4]} {time[5]}"
                            title = x.find('a').text
                            link = x.find('a')['href']

                            # scrap kontennya
                            ge_a = req.get(f"{link}?single=1").text
                            sop_a = bs(ge_a, 'lxml')
                            content = sop_a.find_all('div', class_='detail__body-text itp_bodycontent')

                            for i in content:
                                isi_konten = [x.text for x in i.find_all('p')]
                                try:
                                    writer.writerow([title, tgl_, link, isi_konten])
                                    count += 1
                                    print(f"[{count}][{tgl_}][DETIK] {title}")
                                except:
                                    pass
                except:
                    pass

In [14]:
source_detik(mm= 9, yy= 2024, long_time2 = 1)

[1][01/Sep/2024 23:54 WIB][DETIK] RK Ingin Suasana Pilgub Jakarta Gembira: Tak Boleh Nanti Debat Ngomongin Orang
[2][01/Sep/2024 23:19 WIB][DETIK] Di Mukernas Badan Otonom PKB Berani, Cak Imin Beri Pesan Kebhinnekaan
[3][01/Sep/2024 23:13 WIB][DETIK] Dharma soal Jalur Sepeda: Jangan Paksa Jalan yang Seharusnya Bisa Lewat Mobil
[4][01/Sep/2024 22:35 WIB][DETIK] Dharma Pongrekun-Kun Usung Slogan 'Jakartaku Aman' di Pilgub Jakarta
[5][01/Sep/2024 22:13 WIB][DETIK] Adian Napitupulu Ngaku Yakin PDIP Menang Lawan 17 Parpol di Pilbup Bogor
[6][01/Sep/2024 22:00 WIB][DETIK] Hamas Tunjuk Hidung Israel Usai Serangan Udara Tewaskan Sandera
[7][01/Sep/2024 21:59 WIB][DETIK] 2 Pelaku Rampas Motor Pria di Cikarang, Pura-pura Jadi Polisi Saat Beraksi
[8][01/Sep/2024 21:31 WIB][DETIK] Andika Anggap Persaingan Lawan Luthfi di Jateng Bukan 'Perang Bintang'
[9][01/Sep/2024 21:24 WIB][DETIK] Dharma Pongrekun Minta Warga Ikut Kampanyekan Dirinya di Pilgub Jakarta
[10][01/Sep/2024 21:02 WIB][DETIK] Cerita S

In [18]:
df = pd.read_csv(file_path + 'result_scrape_2024-09-14 17:46:24.csv')

df

,title,released,url,content
0,RK Ingin Suasana Pilgub Jakarta Gembira: Tak B...,01/Sep/2024 23:54 WIB,https://news.detik.com/pilkada/d-7519751/rk-in...,"['Bakal calon Gubernur DKI Jakarta, Ridwan Kam..."
1,"Di Mukernas Badan Otonom PKB Berani, Cak Imin ...",01/Sep/2024 23:19 WIB,https://news.detik.com/berita/d-7519746/di-muk...,['Ketua Umum Partai Kebangkitan Bangsa (PKB) M...
2,Dharma soal Jalur Sepeda: Jangan Paksa Jalan y...,01/Sep/2024 23:13 WIB,https://news.detik.com/pilkada/d-7519745/dharm...,['Bakal calon gubernur Jakarta Dharma Pongreku...
3,Dharma Pongrekun-Kun Usung Slogan 'Jakartaku A...,01/Sep/2024 22:35 WIB,https://news.detik.com/pilkada/d-7519737/dharm...,"[""Pasangan bakal calon gubernur dan wakil gube..."
4,Adian Napitupulu Ngaku Yakin PDIP Menang Lawan...,01/Sep/2024 22:13 WIB,https://news.detik.com/pilkada/d-7519728/adian...,['Wasekjen DPP PDIP Adian Napitupulu menghadir...
...,...,...,...,...
111,Cak Imin Respons Pantun-Ramalan Prabowo: Persa...,01/Sep/2024 02:15 WIB,https://news.detik.com/berita/d-7518709/cak-im...,['Ketum PKB Muhaimin Iskandar atau Cak Imin me...
112,Tentara Israel Tewas dalam Operasi Militer di ...,01/Sep/2024 01:45 WIB,https://news.detik.com/internasional/d-7518705...,['Militer Israel mengumumkan kematian pertama ...
113,Cak Imin soal Ma'ruf Amin Sambangi DPP PKB: Ra...,01/Sep/2024 01:15 WIB,https://news.detik.com/berita/d-7518702/cak-im...,"[""Ketua Umum PKB Muhaimin Iskandar atau Cak Im..."
114,Dasco Jelaskan soal Momen Bertemu Bahlil dan A...,01/Sep/2024 00:37 WIB,https://news.detik.com/pilkada/d-7518674/dasco...,['Ketua Harian Gerindra Sufmi Dasco Ahmad menj...


0